In [1]:
from google.colab import drive

drive.mount('/content/gdriver')

Mounted at /content/gdriver


In [2]:
%cd /content/gdriver/MyDrive/OpenCV

/content/gdriver/MyDrive/OpenCV


In [3]:
!pip install pillow
!pip install opencv-python
!pip install easyocr
!pip install --upgrade sympy

!sudo apt update
!sudo apt install -y tesseract-ocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12066 sha256=43917a4dacc4b1a797aec054694

In [9]:
import PIL
from PIL import Image
import glob
import shutil, os
from time import sleep
import cv2
import numpy as np
import matplotlib.pyplot as mPlt
import easyocr
import re

reader = easyocr.Reader(['en'])

def showImg(img):
    mPlt.imshow(img, cmap='gray')
    mPlt.axis('off')
    mPlt.show()

def emptydir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
        sleep(2)
    os.mkdir(dirname)

def area(row, col):
    global nn
    if bg[row][col] != 255:
        return
    bg[row][col] = lifearea #記錄生命區的編號
    if col>1: #左方
        if bg[row][col-1]==255:
            nn +=1
            area(row,col-1)
    if col< w-1: #右方
        if bg[row][col+1]==255:
            nn +=1
            area(row,col+1)
    if row>1: #上方
        if bg[row-1][col]==255:
            nn+=1
            area(row-1,col)
    if row<h-1: #下方
        if bg[row+1][col]==255:
            nn+=1
            area(row+1,col)


files = glob.glob("test/*.jpg")
dstdir = 'testAns'
emptydir(dstdir)

for (i, file) in enumerate(files):
    filename = file.split(os.sep)[-1]
    fileImg = Image.open(file)
    resizeImg = fileImg.resize((300, 225), PIL.Image.LANCZOS)

    img = np.array(resizeImg)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # showImg(gray)


    detector = cv2.CascadeClassifier('haar_carplate.xml')
    signs = detector.detectMultiScale(img_bgr, scaleFactor=1.1, minNeighbors=4, minSize=(20, 20))

    threImg = None

    if len(signs) > 0:
        for (sx, sy, sw, sh) in signs:

            cv2.rectangle(img_bgr, (sx, sy), (sx + sw, sy + sh), (0, 255, 0), 2)

            image2 = resizeImg.crop((sx, sy, sx + sw, sy + sh))
            image3 = image2.resize((140, 40), Image.LANCZOS)

            imgGray = np.array(image3.convert('L'))
            _, thre_Img = cv2.threshold(imgGray, 127, 255, cv2.THRESH_BINARY_INV)


            threImg = thre_Img

            showImg(imgGray)
            showImg(threImg)

        if threImg is not None:

            thresh = threImg
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


            letter_image_regions = [] #文字圖形串列
            for contour in contours:  #依序處理輪廓
                (x, y, w, h) = cv2.boundingRect(contour)  #單一輪廓資料
                letter_image_regions.append((x, y, w, h)) #輪廓資料加入串列
            letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])  #按X坐標排序

            #先計算可以擷取的字元數
            count=0 #計算共擷取多少個字元
            for box in letter_image_regions:  #依序處理輪廓資料
                x, y, w, h = box
                # x 必須介於 2~125 且寬度在 5~26、高度在 20~39 才是文字
                if x>=2 and x<=125 and w>=5 and w<=26 and h>=20 and h<40:
                    count +=1

            if count<6: #若字元數不足，可能是有兩個字元連在一起，將字元寬度放寬再重新擷取
                wmax=35
            else:
                wmax=26 #正常字元寬度

            nChar=0 #計算共擷取多少個字元
            letterlist = [] #儲存擷取的字元坐標
            for box in letter_image_regions:  #依序處理輪廓資料
                x, y, w, h = box
                # x 必須介於 2~125 且寬度在 5~wmax、高度在 20~39 才是文字
                if x>=2 and x<=125 and w>=5 and w<=wmax and h>=20 and h<40:
                    nChar +=1
                    letterlist.append((x, y, w, h)) #儲存擷取的字元

            #去除雜點
            for i in range(len(thresh)):  #i為高度
                for j in range(len(thresh[i])): #j為寬度
                    if thresh[i][j] == 255:     #顏色為白色
                        count = 0
                        for k in range(-2, 3):
                            for l in range(-2, 3):
                                try:
                                    if thresh[i + k][j + l] == 255: #若是白點就將count加1
                                        count += 1
                                except IndexError:
                                    pass
                        if count <= 6:  #週圍少於等於6個白點
                            thresh[i][j] = 0  #將白點去除

            real_shape=[]
            for i,box in enumerate(letterlist):  #依序擷取所有的字元
                x, y, w, h = box
                bg=thresh[y:y+h, x:x+w]

                # 去除崎鄰地
                if i==0 or i==nChar: # 只去除第一字元和最後字元的崎鄰地
                    lifearea=0 # 生命區塊
                    nn=0       # 每個生命區塊的生命數
                    life=[]    # 記錄每個生命區塊的生命數串列
                    for row in range(0,h):
                        for col in range(0,w):
                          if bg[row][col] == 255:
                              nn = 1  #生命起源
                              lifearea = lifearea + 1  #生命區塊數
                              area(row,col)  #以生命起源為起點探索每個生命區塊的總生命數
                              life.append(nn)

                    maxlife=max(life) #找到最大的生命數
                    indexmaxlife=life.index(maxlife) #找到最大的生命數的區塊編號

                    for row in range(0,h):
                        for col in range(0,w):
                          if bg[row][col] == indexmaxlife+1:
                              bg[row][col]=255
                          else:
                              bg[row][col]=0

                real_shape.append(bg) #加入字元

            #在圖片週圍加白色空白OCR才能辨識
            newH, newW = thresh.shape

            space = 8  #空白寬度
            offset=2
            bg = np.zeros((newH+space*2, newW+space*2+nChar*3, 1), np.uint8)  #建立背景
            bg.fill(0)  #背景黑色

            # 將車牌文字加入黑色背景圖片中
            for i,letter in enumerate(real_shape):
                h=letter.shape[0]   #原來文字圖形的高、寬
                w=letter.shape[1]
                x=letterlist[i][0]  #原來文字圖形的位置
                y=letterlist[i][1]
                for row in range(h):#將文字圖片加入背景
                    for col in range(w):
                        bg[space+y+row][space+x+col+i*offset] = letter[row][col] #擷取圖形

            _,bg = cv2.threshold(bg, 127, 255, cv2.THRESH_BINARY_INV)

            showImg(bg)

            scale_percent = 150
            width = int(bg.shape[1] * scale_percent / 100)
            height = int(bg.shape[0] * scale_percent / 100)
            dim = (width, height)
            resized_image = cv2.resize(bg, dim, interpolation=cv2.INTER_AREA)


            result = reader.readtext(resized_image)

            if result:
                text = re.sub('[^A-Za-z0-9]', '',result[0][1].replace("|", "1").replace("/", "1").replace(' ', '')).upper()

                print(filename)
                print("辨識結果：", text)

                text_x_position = sx + sw // 2 - (cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)[0][0] // 2)
                cv2.putText(img_bgr, text, (text_x_position, sy + sh + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                filename_to_save = dstdir + '/' + text + '.jpg'
                cv2.imwrite(filename_to_save, img_bgr) #存檔



                mPlt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
                mPlt.axis('off')
                mPlt.show()
                print("---------------------------------------------------")


            else:
                print("無法辨識文字的圖片：", filename)


    else:
        print('無法擷取車牌的圖片：', filename)


Output hidden; open in https://colab.research.google.com to view.